In [ ]:
# Install libek
install.packages("readxl")
install.packages("dplyr")
install.packages("openxlsx")
install.packages("rstatix")
install.packages("tidyverse")
install.packages("moments")

In [2]:
# load libek
library(dplyr)
library(tidyverse)
library(moments)
library(rstatix)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.1     ✔ readr     2.1.5
✔ ggplot2   4.0.0     ✔ stringr   1.5.2
✔ lubridate 1.9.4     ✔ tibble    3.3.0
✔ purrr     1.1.0     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter




In [3]:
# Načtení dat z excelu -> Název souboru ||     Sheet         || skip řádků
data <- readxl::read_excel("data_hraci.xlsx", sheet = "hraci", skip = 0)
head(data)

ERROR: Error: `path` does not exist: ‘data_hraci.xlsx’


In [ ]:
# POKUD DATA POTŘEBUJI PREPROCESOVAT, NEMÁM JE TREBA ROZDĚLENÉ...

# ==============================================================================
# 🔄 UNIVERZÁLNÍ PIVOT (WIDE -> LONG) - TAHÁK
# ==============================================================================
# Kdy použít: Když máš měření ve sloupcích (rok_2018, rok_2019) a chceš je pod sebe.
# Co upravit:
#   1. cols -> Které sloupce se mají "smrsknout" (ty co obsahují měřená data)
#   2. names_to -> Název nového sloupce pro kategorii (např. "Rok", "Mesic")
#   3. values_to -> Název nového sloupce pro hodnoty (např. "Pocet", "Vykon")
#   4. names_prefix -> (Volitelné) Text, který chceš umazat z názvu sloupců
# ==============================================================================

# Předpoklad: data jsou načtená v proměnné 'data'

df_long <- data %>%
  pivot_longer(
    # --- 1. VÝBĚR SLOUPCŮ (Odkomentuj jen jednu možnost) ---

    # Možnost A: Vyjmenovat konkrétní sloupce (NEJBEZPEČNĚJŠÍ)
    # Použij, když je sloupců málo a víš přesně, jak se jmenují.
    cols = c(odehrane_hod_2018, odehrane_hod_2019),
  
    # Možnost B: Všechno KROMĚ identifikačních sloupců (NEJRYCHLEJŠÍ)
    # Použij, když máš mraky sloupců s daty a jen pár IDček (ID, jméno, system).
    # cols = -c(IDhrace, system),
    
    # Možnost C: Sloupce začínající určitým textem (CHYTRÉ)
    # cols = starts_with("odehrane_hod_"),

    # --------------------------------------------------------
    
    # --- 2. NASTAVENÍ NÁZVŮ ---
    names_to = "Rok",       # Jak nazvat sloupec, kam spadnou "2018", "2019"
    values_to = "Hodiny",   # Jak nazvat sloupec, kam spadnou čísla
    
    # --- 3. ČIŠTĚNÍ NÁZVŮ (Odkomentuj podle potřeby) ---
    
    # Situace A: Jednoduchý prefix (Tvůj aktuální případ)
    # Máš "odehrane_hod_2018" -> chceš jen "2018".
    names_prefix = "odehrane_hod_"
    
    # Situace B: Složitější vzor (Regulární výrazy) - PROFI
    # Pokud bys měl např. "Karta_Nvidia_FPS" a chtěl vytáhnout Kartu i Metriku.
    # names_pattern = "(.*)_average_(.*)", 
    # names_to = c("Karta", "Metrika") 
  ) 

# --- 4. DODATEČNÉ ÚPRAVY (Volitelné) ---

# Převod Roku na číslo (protože z názvu sloupce se vždy stane text)
# df_long$Rok <- as.numeric(df_long$Rok)

# Kontrola výsledku
head(df_long)

ERROR: Error in data %>% pivot_longer(cols = c(odehrane_hod_2018, odehrane_hod_2019), : could not find function "%>%"


In [ ]:
# ==============================================================================
# 🕵️‍♂️ DETEKCE ODLEHLÝCH HODNOT (Vnitřní hradby / Boxplot) - TAHÁK
# ==============================================================================
# Kdy použít: "Identifikujte odlehlá pozorování...", "Vypočtěte kvartil po odstranění..."
# Postup:
#   1. Zkontroluj názvy kategorií (aby seděl filtr).
#   2. Vyfiltruj data (např. jen "WIN" nebo rok "2018").
#   3. Vypočti Q1, Q3, IQR a Hradby (Meze).
#   4. Najdi odlehlé a odstraň je.
# ==============================================================================


# --- 0. RYCHLÁ KONTROLA KATEGORIÍ (Spusť v konzoli předem) ---
# unique(data$system)  # Zjistíš, jestli je to "Windows", "WIN" nebo "W"

# --- 1. PŘÍPRAVA DAT (Uprav jen názvy sloupců a hodnotu filtru) ---
data_subset <- data %>%
  # A) FILTRACE (Pokud potřebuješ jen část dat)
  # filter(system == "WIN") %>% 
  
  # B) VÝBĚR SLOUPCE (Vyber ten jeden číselný, co zkoumáš + ID pro výpis)
  select(ID = IDhrace, Hodnota = odehrane_hod_2018) %>% 
  
  # C) OČIŠTĚNÍ (Vyhodí prázdné buňky, aby nepadaly výpočty)
  drop_na()

# --- 2. VÝPOČET HRANIC (Vnitřní hradby) ---
# na.rm = TRUE je pojistka, kdyby drop_na výše nezafungovalo
Q1  <- quantile(data_subset$Hodnota, 0.25, na.rm = TRUE)
Q3  <- quantile(data_subset$Hodnota, 0.75, na.rm = TRUE)
IQR_val <- IQR(data_subset$Hodnota, na.rm = TRUE)

DM <- Q1 - 1.5 * IQR_val # Dolní mez
HM <- Q3 + 1.5 * IQR_val # Horní mez

print(paste("Hradby: <", round(DM, 2), ";", round(HM, 2), ">"))

# --- 3. IDENTIFIKACE ODLEHLÝCH (Kdo je venku?) ---
odlehle <- data_subset %>%
  filter(Hodnota < DM | Hodnota > HM)

print("--- ODLEHLÁ POZOROVÁNÍ (Opiš ID a Hodnoty) ---")
print(odlehle)

# --- 4. VÝPOČET STATISTIK PO ODSTRANĚNÍ ODLEHLÝCH ---
data_clean <- data_subset %>%
  filter(Hodnota >= DM & Hodnota <= HM)

# Tady si odkomentuj, co po tobě chtějí spočítat z čistých dat:
vysledek <- quantile(data_clean$Hodnota, 0.75, na.rm = TRUE) # Nový horní kvartil
# vysledek <- mean(data_clean$Hodnota, na.rm = TRUE)         # Nový průměr
# vysledek <- sd(data_clean$Hodnota, na.rm = TRUE)           # Nová směr. odchylka

print(paste("Výsledek z čistých dat:", round(vysledek, 2)))

In [ ]:
# ==============================================================================
# 📊 VÝPOČET VŠECH METRIK (Popisná statistika) - TAHÁK
# ==============================================================================
# Kdy použít: "Určete střední hodnotu, rozptyl, kvartily, variační koeficient..."
# ==============================================================================

# 1. VÝBĚR DAT (Tady jen změň název datasetu a sloupce!)
# ------------------------------------------------------
# Příklad: vybírám sloupec 'Vynos_A' z tabulky 'data'
# Uložím si to do proměnné 'x', abych nemusel pořád přepisovat názvy dole
x <- data$Rocni_Vynosnost_v_Procentech 

# (Volitelně) Pokud chceš jen konkrétní skupinu (např. Strategie A):
# x <- data %>% filter(Strategie == "A") %>% pull(Rocni_Vynosnost_v_Procentech)


# 2. VÝPOČET METRIK (Tohle už jen spusť celé najednou)
# ------------------------------------------------------

# --- MÍRY POLOHY (Střed) ---
prumer  <- mean(x, na.rm = TRUE)       # Střední hodnota (E(X), mu)
median_x <- median(x, na.rm = TRUE)    # Medián (prostřední hodnota, 50% kvantil)

# --- MÍRY VARIABILITY (Jak moc to létá) ---
rozptyl <- var(x, na.rm = TRUE)        # Rozptyl (s^2, Variance)
smer_odchylka <- sd(x, na.rm = TRUE)   # Směrodatná odchylka (sigma, SD)
rozpeti <- max(x, na.r = TRUE) - min(x, na.rm = TRUE) # Variační rozpětí (Max - Min)

# Variační koeficient (CV) - v procentech!
# Vzorec: (Směrodatná odchylka / Průměr) * 100
var_koef <- (smer_odchylka / abs(prumer)) * 100 

# --- KVANTILY (Percentily) ---
# min, 25% (Q1), 50% (Medián), 75% (Q3), max
kvantily <- quantile(x, probs = c(0, 0.25, 0.5, 0.75, 1), na.rm = TRUE)
IQR_hodnota <- IQR(x, na.rm = TRUE)    # Mezikvartilové rozpětí (Q3 - Q1)

# 3. VÝPIS VÝSLEDKŮ
# ------------------------------------------------------
print("--- ZÁKLADNÍ METRIKY ---")
print(paste("Průměr (Mean):", round(prumer, 4)))
print(paste("Medián:", round(median_x, 4)))

print("--- VARIABILITA ---")
print(paste("Rozptyl (Var):", round(rozptyl, 4)))
print(paste("Směrodatná odchylka (SD):", round(smer_odchylka, 4)))
print(paste("Variační koeficient (CV):", round(var_koef, 2), "%"))

print("--- KVANTILY A HRADBY ---")
print(kvantily)
print(paste("IQR (Q3-Q1):", round(IQR_hodnota, 4)))

ERROR: Error in data$Rocni_Vynosnost_v_Procentech: object of type 'closure' is not subsettable


In [ ]:
# ==============================================================================
# 📐 INTERVALOVÝ ODHAD PODÍLU (prop.test) - TAHÁK
# ==============================================================================
# 1. Zjisti typ intervalu ze zadání (nebo podle hypotézy)
#    - "Oboustranný" (se rovná/nerovná) -> alternative = "two.sided"
#    - "Větší než" (Pravostranný test)   -> alternative = "greater"  (Hledáme Dolní mez)
#    - "Menší než" (Levostranný test)    -> alternative = "less"     (Hledáme Horní mez)

# 2. Nastav hladinu spolehlivosti (conf.level)
#    - 95% -> 0.95
#    - 90% -> 0.90

# 3. Kód (Příklad pro "Větší než", 90%)
test_int <- prop.test(x = x_A, n = n_A, conf.level = 0.90, alternative = "greater")

print(paste("Interval: <", round(test_int$conf.int[1], 4), ";", round(test_int$conf.int[2], 4), ">"))